In [ ]:
%load_ext watermark

import os

import uxarray as ux
import xarray as xr

%watermark -iv

In [ ]:
# Base data path
base_path = os.path.expanduser("~/work/python/uxarray/test/meshfiles/ugrid/geoflow-small/")

# Path to Grid file
grid_path = base_path + "grid.nc"

# Paths to Data Variable files
var_names = ["v1.nc", "v2.nc", "v3.nc"]
data_paths = [base_path + name for name in var_names]

# Open the Grid file
grid_ds = xr.open_dataset(grid_path)

# Open a single file or multiple files
v1_ds = xr.open_dataset(data_paths[0])
multi_data_ds = xr.open_mfdataset(data_paths)

In [ ]:
# Construct a UXarray Grid object from `grid_ds`
grid = ux.Grid(grid_ds)
grid

In [ ]:
grid_ds

In [ ]:
from xarray.core.indexes import Index, PandasIndex, get_indexer_nd
from xarray.core.indexing import merge_sel_results


class UGridIndex(Index):
    # based off Benoit's RasterIndex in
    # https://hackmd.io/Zxw_zCa7Rbynx_iJu6Y3LA?view

    def __init__(self, variables):
        # TODO: hardcoded variable names

        # assert len(xy_indexes) == 2
        assert "x" in variables
        assert "y" in variables
        assert "spatial_ref" in variables

        # TODO: Instead do whatever the rio accessor is doing.
        # rioxarray.open_dataset is doing
        spatial_ref = variables.pop("spatial_ref")
        self._crs = rioxarray.crs.CRS.from_wkt(spatial_ref.attrs["crs_wkt"])

        # must have two distinct dimensions
        # Assumes x, y for index are never scalar. Is that correct?
        dim = [idx.dim for key, idx in variables.items()]
        assert dim[0] != dim[1]

        self._indexes = variables

    # TODO: what goes in options?
    @classmethod
    def from_variables(cls, variables, options):
        # assert len(variables) == 2

        xy_indexes = {
            k: PandasIndex.from_variables({k: v}, options=options)
            for k, v in variables.items()
            if k in ["x", "y"]
        }
        xy_indexes["spatial_ref"] = variables["spatial_ref"]

        return cls(xy_indexes)

    # TODO: variables=None?
    # set_xindex tries to pass variables; this seems like a bug
    def create_variables(self, variables=None):
        idx_variables = {}

        for index in self._indexes.values():
            idx_variables.update(index.create_variables(variables))

        idx_variables["spatial_ref"] = create_spatial_ref(self.as_wkt)
        return idx_variables

    # TODO: see notes about IndexSelResult
    #    The latter is a small class that stores positional indexers (indices)
    #    and that could also store new variables, new indexes,
    #    names of variables or indexes to drop,
    #    names of dimensions to rename, etc.
    def sel(self, labels, **kwargs):
        # sel needs to only handle keys in labels
        # since it delegates to isel.
        # we handle all entries in ._indexes there
        results = []
        for k, index in self._indexes.items():
            if k in labels:
                # defer to pandas type indexing.
                # This is where we would implement KDTree and friends
                results.append(index.sel({k: labels[k]}, **kwargs))
        return merge_sel_results(results)

    def isel(self, indexers):
        # TODO: check dim names in indexes
        results = {}
        for k, index in self._indexes.items():
            if k in indexers:
                # again possible KDTree / friends here.
                results[k] = index.isel({k: indexers[k]})
            else:
                results[k] = index
        # AGAIN!
        results["spatial_ref"] = create_spatial_ref(self.as_wkt)
        return type(self)(results)

    def __repr__(self):
        string = f"CRSIndex: {self._crs.to_string()}"
        return string

    def equals(self, other):
        result = self._crs is other._crs or (
            self._crs == other._crs
            and self._indexes["x"].equals(other._indexes["x"])
            and self._indexes["y"].equals(other._indexes["y"])
        )
        return result

    def join(self, other, how="inner"):
        if self._crs != other._crs:
            raise ValueError(
                "Cannot align or join objects with different CRS. "
                f"Received {self._crs.name!r} and {other._crs.name!r}"
            )

        new_indexes = {k: v.join(other._indexes[k], how=how) for k, v in self._indexes.items()}
        # create new spatial_ref here.
        new_indexes["spatial_ref"] = create_spatial_ref(self.as_wkt)
        return type(self)(new_indexes)

    def reindex_like(self, other, method=None, tolerance=None):
        # TODO: different method, tolerance for x, y?
        return {
            k: get_indexer_nd(self._indexes[k].index, other._indexes[k].index, method, tolerance)
            for k in self._indexes.keys()
        }

    @property
    def as_crs(self):
        return self._crs

    @property
    def as_wkt(self):
        return self._crs.to_wkt()

In [ ]:
ux.Grid

In [ ]:
merged = xr.merge([v1_ds, grid_ds])
merged